## Import library

In [1]:
import pandas as pd
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')
import test_results
import data_load
import numpy as np
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import sklearn as sk

from imblearn.over_sampling import SMOTE
from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


## Load data

In [2]:
df = pd.read_csv("/home/dtd/Downloads/Data_Scientist_Nanodegree/starbucks_portfolio_exercise/training.csv")

In [3]:
df.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [4]:
df['treatment'] = np.where(df['Promotion'] == "Yes", 1, 0)

In [5]:
df.purchase.value_counts()

0    83494
1     1040
Name: purchase, dtype: int64

In [6]:
col = list(df.columns)

treatment = 'treatment' # intervention, or treatment
outcome = 'purchase' # outcome of interest

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove("ID")
cov.remove("Promotion")
print(cov)

features = col[:]
features.remove(outcome)
features.remove("ID")
features.remove("Promotion")
print(features)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']
['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'treatment']


In [7]:
X = df[features]
y = df[outcome]

In [8]:
y.value_counts()

0    83494
1     1040
Name: purchase, dtype: int64

In [9]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_sampling, y_sampling = ros.fit_resample(X, y)

In [10]:
data = pd.concat([X_sampling, y_sampling], axis=1, sort=False)

In [11]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,treatment,purchase
0,2,30.443518,-1.165083,1,1,3,2,0,0
1,3,32.159350,-0.645617,2,3,2,2,0,0
2,2,30.431659,0.133583,1,1,4,2,0,0
3,0,26.588914,-0.212728,2,1,4,2,0,0
4,3,28.044332,-0.385883,1,1,2,2,1,0


In [12]:
data[cov]

,V1,V2,V3,V4,V5,V6,V7
0,2,30.443518,-1.165083,1,1,3,2
1,3,32.159350,-0.645617,2,3,2,2
2,2,30.431659,0.133583,1,1,4,2
3,0,26.588914,-0.212728,2,1,4,2
4,3,28.044332,-0.385883,1,1,2,2
...,...,...,...,...,...,...,...
166983,1,39.167312,-0.732194,2,2,1,1
166984,0,31.698306,0.133583,2,1,1,2
166985,1,37.086413,1.605406,2,2,4,2
166986,1,26.179611,-0.818772,2,3,3,2


## Calculate treatment effect 

In [13]:
model_t = LogisticRegression()
model_t.fit(data[cov].values, data[treatment].values)

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 50)
model_y.fit(data[features], data[outcome])

GradientBoostingRegressor(n_estimators=50, random_state=0)

In [14]:
data['p1'] = model_t.predict_proba(data[cov].values)[:,:1]
data['p0'] = 1 - model_t.predict_proba(data[cov].values)[:,:1]
data['predicted_t'] = [1 if i >= 0.5 else 0 for i in data['p1']]

In [15]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,treatment,purchase,p1,p0,predicted_t
0,2,30.443518,-1.165083,1,1,3,2,0,0,0.495269,0.504731,0
1,3,32.159350,-0.645617,2,3,2,2,0,0,0.358259,0.641741,0
2,2,30.431659,0.133583,1,1,4,2,0,0,0.524033,0.475967,1
3,0,26.588914,-0.212728,2,1,4,2,0,0,0.386798,0.613202,0
4,3,28.044332,-0.385883,1,1,2,2,1,0,0.510114,0.489886,1


## Optimization
Find the delta and promotion strategy in order to maximize irr, nir

In [16]:
def score(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return (irr, nir)

## Test result

In [42]:
# testing baseline model, where we send promotions to everyone

def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]            
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    test = df
    promotion = []
    num_test_points = test.shape[0]
    for i in range(num_test_points):
        promotion.append('Yes')
    promotion = np.array(promotion)
    return promotion

def score_(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return (irr, nir)

def results(promotion_strategy, data):
    df = data[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
    promos = promotion_strategy(df)
    score_df = data.iloc[np.where(promos == 'Yes')]
    irr, nir = score_(score_df)
    return irr, nir


## Baseline model 

In [18]:
df = pd.read_csv("/home/dtd/Downloads/Data_Scientist_Nanodegree/starbucks_portfolio_exercise/training.csv")
train_results_(promotion_strategy, df)

In [19]:
df['p1'] = model_t.predict_proba(df[cov].values)[:,:1]
df['p0'] = 1 - model_t.predict_proba(df[cov].values)[:,:1]
df['predicted_t'] = [1 if i >= 0.5 else 0 for i in df['p1']]

In [23]:
df.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7,p1,p0,predicted_t,q1,incre_q1,proposed_strategy
0,1,No,0,2,30.443518,-1.165083,1,1,3,2,0.495269,0.504731,0,0.443594,0,No
1,3,No,0,3,32.159350,-0.645617,2,3,2,2,0.358259,0.641741,0,0.214112,0,No
2,4,No,0,2,30.431659,0.133583,1,1,4,2,0.524033,0.475967,0,NaN,0,No
3,5,No,0,0,26.588914,-0.212728,2,1,4,2,0.386798,0.613202,0,0.483042,0,No
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2,0.510114,0.489886,0,NaN,0,No


## Explore the changes 

In [1]:
from collections import Counter
delta_seq = np.random.normal(0.1, 10, 1000)
rev_seq = []
dictionary_ = []
delta_result = []

df['proposed_strategy'] = ''

for delta in tqdm(delta_seq):    
    q1 = (delta * df['p1']) / (delta * df['p1'] + df['p0'])
    policy = np.array([1 if i >= 0.5 else 0 for i in q1])
    df['proposed_strategy'] = [ "Yes" if i == 1 else "No" for i in policy]
    d = dict(Counter(policy))
    count = 0
    if 1 not in d:
        count = 0
    else:
        count = d[1]
    irr, nir = score_(df, promo_pred_col = 'proposed_strategy')
    
    if not math.isnan(nir):
        delta_result.append(delta)
        rev_seq.append(nir)
        if count not in dictionary_:
            dictionary_.append(count)
            print("Number of treatment {}".format(count))
            print("Revenue {}".format(nir))
            print("*"*100)

NameError: name 'np' is not defined

In [28]:
max(rev_seq)

## Train test

In [50]:
# read data
test_data = pd.read_csv('/home/dtd/Downloads/Data_Scientist_Nanodegree/starbucks_portfolio_exercise/Test.csv')
train_data = pd.read_csv('/home/dtd/Downloads/Data_Scientist_Nanodegree/starbucks_portfolio_exercise/training.csv')

In [41]:
score(test_data, promo_pred_col = 'Promotion')

In [52]:
test_data['p1'] = model_t.predict_proba(test_data[cov].values)[:,:1]
test_data['p0'] = 1 - model_t.predict_proba(test_data[cov].values)[:,:1]

In [ ]:
from collections import Counter
delta_seq = np.random.uniform(low=0.1, high=10, size=(10000,))
rev_seq = []
dictionary_ = []
delta_result = []

test_data['proposed_strategy'] = ''

for delta in tqdm(delta_seq):    
    q1 = (delta * test_data['p1']) / (delta * test_data['p1'] + test_data['p0'])
    policy = np.array([1 if i >= 0.5 else 0 for i in q1])
    test_data['proposed_strategy'] = [ "Yes" if i == 1 else "No" for i in policy]
    d = dict(Counter(policy))
    count = 0
    if 1 not in d:
        count = 0
    else:
        count = d[1]
    irr, nir = score_(test_data, promo_pred_col = 'proposed_strategy')
    
    if not math.isnan(nir):
        delta_result.append(delta)
        rev_seq.append(nir)
        if count not in dictionary_:
            dictionary_.append(count)
            print("Number of treatment {}".format(count))
            print("Revenue {}".format(nir))
            print("*"*100)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys
  0%|          | 3/10000 [00:00<07:43, 21.58it/s]

41650 0 480 0
Number of treatment 41650
Revenue -1447.5
****************************************************************************************************
41648 2 480 0
Number of treatment 41648
Revenue -1447.1999999999998
****************************************************************************************************
41650 0 480 0
41650 0 480 0
41650 0 480 0


  0%|          | 9/10000 [00:00<07:13, 23.03it/s]

41650 0 480 0
41650 0 480 0
41650 0 480 0
41650 0 480 0
37303 4347 403 77
Number of treatment 37303
Revenue -2335.45
****************************************************************************************************


  0%|          | 15/10000 [00:00<06:59, 23.78it/s]

...

 87%|████████▋ | 8715/10000 [06:08<00:53, 23.84it/s]

41650 0 480 0
41650 0 480 0
41650 0 480 0
41650 0 480 0
41650 0 480 0


 87%|████████▋ | 8721/10000 [06:08<00:53, 23.98it/s]

41650 0 480 0
41650 0 480 0
41650 0 480 0
41650 0 480 0
38665 2985 429 51
Number of treatment 38665
Revenue -2019.75
****************************************************************************************************


 87%|████████▋ | 8724/10000 [06:08<00:55, 22.99it/s]

In [37]:
import pickle
with open ('/home/dtd/Downloads/Data_Scientist_Nanodegree/starbucks_portfolio_exercise/outfile.txt', 'rb') as fp:
    itemlist = pickle.load(fp)

In [38]:
itemlist

array(['No', 'Yes', 'No', ..., 'No', 'No', 'Yes'], dtype='<U3')

In [39]:
policy = np.array([1 if i >= 0.5 else 0 for i in q1])
valid['proposed_strategy'] = itemlist
d = dict(Counter(policy))
count = 0
if 1 not in d:
    count = 0
else:
    count = d[1]
irr, nir = score_(valid, promo_pred_col = 'proposed_strategy')

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
